In [50]:
import geopandas as gpd
import pandas as pd
import fiona
from shapely.geometry import shape
from shapely.geometry import MultiLineString
import shapely
import os

In [2]:
#gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
fiona.drvsupport.supported_drivers['kml'] = 'rw'  # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw'  # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'  # enable KML support which is disabled by default

In [3]:
file = r"C:\Users\pnjoya\Desktop\01_IT Data\historical_data\tracks_archives\africa\diana\w17\W17_track_V1.0.gpx"
file2 = r"C:\Users\pnjoya\Desktop\01_IT Data\historical_data\tracks_archives\europe\espana organica\W13\7-45-186-5-157-R1_W13.kml"
file3 = r"C:\Users\pnjoya\Desktop\98_R&D\insert_historical_data\test_multi.kml"

## GPX

In [46]:
##GPX##

#find the right gpx layer
list_layers = fiona.listlayers(file)

list_size_layers = list(len(list(fiona.open(file, layer = x))) for x in list_layers)

min_value = min(x for x in list_size_layers if not x==0)

layer_position = list_size_layers.index(min_value)

layer_selected = list_layers[layer_position]

#GPX to dataframe
gps = fiona.open(file, layer = layer_selected)
gps_list = list(gps)

#Transform geometry type in MultiLineString
for i in gps_list :
    geom = [[]]
    for y in i["geometry"]["coordinates"]:
        if isinstance(y, list):
            for z in y:
                point = z[:2]
                geom[0].append(point)
        elif isinstance(y, tuple):
            point = y[:2]
            geom[0].append(point)
        else :
            print("probleme here")
    i["geometry"]["coordinates"] = geom
    i["geometry"]["type"] = 'MultiLineString'

#Create dataFrame
df = pd.DataFrame(gps_list)

#Clean DataFrame
keys = list(df.properties[0].keys())
keys = [x.lower() for x in keys]
values = list(df.properties[0].values())

df[keys] = pd.DataFrame(df.properties.tolist(), index = df.index)


df['num_points'] = df['geometry'].apply(lambda x: len(x['coordinates'][0])) #Get number of points per tracks

df_numpoints = df.copy()[df.num_points >= 2] #Filter to have only tracks with more (or equal) than 2 points

df_numpoints.geometry = df_numpoints.geometry.apply(lambda x : shape(x)) #Transform geometry dictionnary into a shapely object

df_numpoints = df_numpoints[["id","geometry","name"]]

#DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df_numpoints) #Transform DataFrame intot a GeoDataFrame

gdf = gdf[["id","geometry","name"]] #Select columns only

## KML

In [44]:
##KML##
kml = fiona.open(file2)
kml_list = list(kml)

for i in kml_list :
    geom = [[]]
    for y in i["geometry"]["coordinates"]:
        if isinstance(y, list):
            for z in y:
                point = z[:2]
                geom[0].append(point)
        elif isinstance(y, tuple):
            point = y[:2]
            geom[0].append(point)
        else :
            print("probleme here")
    i["geometry"]["coordinates"] = geom
    i["geometry"]["type"] = 'MultiLineString'
    
df = pd.DataFrame(kml_list)

#Clean DEataFrame
keys = list(df.properties[0].keys())
keys = [x.lower() for x in keys]
values = list(df.properties[0].values())

df[keys] = pd.DataFrame(df.properties.tolist(), index = df.index)


df['num_points'] = df['geometry'].apply(lambda x: len(x['coordinates'][0])) #Get number of points per tracks

df_numpoints = df.copy()[df.num_points >= 2] #Filter to have only tracks with more (or equal) than 2 points

df_numpoints.geometry = df_numpoints.geometry.apply(lambda x : shape(x)) #Transform geometry dictionnary into a shapely object

df_numpoints = df_numpoints[["id","geometry","name"]]

#DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df_numpoints) #Transform dataFrame intot a geoDataFrame

gdf = gdf[["id","geometry","name"]] #Select columns only

In [47]:
kml = fiona.open(file3)

kml_list = list(kml)

for i in kml_list :
    geom = [[]]
    for y in i["geometry"]["coordinates"]:
        if isinstance(y, list):
            for z in y:
                point = z[:2]
                geom[0].append(point)
        elif isinstance(y, tuple):
            point = y[:2]
            geom[0].append(point)
        else :
            print("probleme here")
    i["geometry"]["coordinates"] = geom
    i["geometry"]["type"] = 'MultiLineString'
    
df = pd.DataFrame(kml_list)

#Clean DEataFrame
keys = list(df.properties[0].keys())
keys = [x.lower() for x in keys]
values = list(df.properties[0].values())

df[keys] = pd.DataFrame(df.properties.tolist(), index = df.index)


df['num_points'] = df['geometry'].apply(lambda x: len(x['coordinates'][0])) #Get number of points per tracks

df_numpoints = df.copy()[df.num_points >= 2] #Filter to have only tracks with more (or equal) than 2 points

df_numpoints.geometry = df_numpoints.geometry.apply(lambda x : shape(x)) #Transform geometry dictionnary into a shapely object

df_numpoints = df_numpoints[["id","geometry","name"]]

#DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df_numpoints) #Transform dataFrame intot a geoDataFrame

gdf = gdf[["id","geometry","name"]] #Select columns only

In [59]:
folder = r"C:\Users\pnjoya\Desktop\01_IT Data\historical_data\tracks_archives\africa\ivory coast\W20"

for root, dirs, files in os.walk(folder) :
    for files in os.listdir(root) :
        filename = files
        print(filename)
        gpx_test = files.lower().endswith('.gpx')
        xml_test = files.lower().endswith('.xml')
        kml_test = files.lower().endswith('.kml')
        file = root + "\\" + files
        
        if gpx_test :
            #find the right gpx layer
            list_layers = fiona.listlayers(file) #list all layers for GPX file
            list_size_layers = list(len(list(fiona.open(file, layer = x))) for x in list_layers) #Get the number of elements of each layers
            min_value = min(x for x in list_size_layers if not x==0) #Find the minimum value that is not null
            layer_position = list_size_layers.index(min_value) #Get the the position of this minimum
            layer_selected = list_layers[layer_position] #Get the layer that has this minimum value

            #GPX to dataframe
            geom_file = fiona.open(file, layer = layer_selected)
        
        elif kml_test:
            geom_file = fiona.open(file)
            
        file_list = list(geom_file)
        
        for i in file_list :
            geom = [[]]
            for y in i["geometry"]["coordinates"]:
                if isinstance(y, list):
                    for z in y:
                        point = z[:2]
                        geom[0].append(point)
                elif isinstance(y, tuple):
                    point = y[:2]
                    geom[0].append(point)
                else :
                    print("probleme here")
            i["geometry"]["coordinates"] = geom
            i["geometry"]["type"] = 'MultiLineString'

        df = pd.DataFrame(file_list)

        #Clean DEataFrame
        keys = list(df.properties[0].keys())
        keys = [x.lower() for x in keys]
        values = list(df.properties[0].values())

        df[keys] = pd.DataFrame(df.properties.tolist(), index = df.index)


        df['num_points'] = df['geometry'].apply(lambda x: len(x['coordinates'][0])) #Get number of points per tracks

        df_numpoints = df.copy()[df.num_points >= 2] #Filter to have only tracks with more (or equal) than 2 points

        df_numpoints.geometry = df_numpoints.geometry.apply(lambda x : shape(x)) #Transform geometry dictionnary into a shapely object

        df_numpoints = df_numpoints[["id","geometry","name"]]
        
        #DataFrame to GeoDataFrame
        gdf = gpd.GeoDataFrame(df_numpoints) #Transform dataFrame intot a geoDataFrame

        gdf = gdf[["id","geometry","name"]] #Select columns only
        
        print(gdf.head(2))

CABB.gpx
  id                                           geometry  \
0  0  MULTILINESTRING ((-3.55241 6.11807, -3.55241 6...   
1  1  MULTILINESTRING ((-3.55601 6.11455, -3.55606 6...   

                                        name  
0       REG__IND_W20_ABBECHOLUCIEIDA__KKO_P1  
1  REG__IND_W20_ABBEGBOCHOFRANÇIS__KKO_P1001  
CAG.gpx
  id                                           geometry  \
0  0  MULTILINESTRING ((-7.52387 6.65433, -7.52385 6...   
1  1  MULTILINESTRING ((-7.49715 6.73703, -7.49720 6...   

                                         name  
0     REG__GUE_W20_AHOUKOUADIOFRANCIS__KKO_P1  
1  REG__GUE_W20_AHOUTOUKOUADIOLAURENT__KKO_P1  
COOPADI.gpx
  id                                           geometry  \
0  0  MULTILINESTRING ((-7.11889 6.93973, -7.11884 6...   
1  1  MULTILINESTRING ((-7.07520 6.90065, -7.07529 6...   

                                     name  
0            REG__GUE_W20_AHOUYAO__KKO_P1  
1  REG__GUE_W20_ALLAGBAKOFFIJULES__KKO_P1  
COOPAWEB.gpx
  id   